In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import modulo as md
import re
from functools import reduce


In [3]:
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}

In [4]:
url='https://fbref.com/es/equipos/361ca564/2022-2023/all_comps/Estadisticas-de-Cadiz-Todas-las-competencias'
res = requests.get(url, headers=headers)
soup=BeautifulSoup(res.text, 'html.parser')


# Aca esta la funcion unificada

In [35]:
def get_players_mean(res,url):# ejemplo url https://fbref.com/es/equipos/b8fd03ef/2022-2023/all_comps/Estadisticas-de-Manchester-City-Todas-las-competencias
    
    codigoE=url.split('equipos/')[1].split('/20')[0]
    soup=BeautifulSoup(res.text, 'html.parser')
    pagina=pd.read_html(url)
    if len(pagina) <30:

        return pd.DataFrame()

    ##BLOQUE 1 
    tablas=[]
    tablas.append(pagina[0])# elijo la tabla 0 que son las estadisticas estandar
    # considero en cuantas competiciones participa el equipo para luego poder elegir las tablas 
    tamaño=soup.find('div', class_='filter switcher')
    '''
    EStas son las tablas que usaré. Las posiciones varian segun las competencias en las que el equipo haya participado, pero suponiendo que solo participó en liga,
    estas serian las posiciones
    estandar=pagina[0], porteros=pagina[2], porteros_avanzada=pagina[3],tiros=pagina[4],pases=pagina[5],creacion=pagina[7],acciones_defensivas=pagina[8], 
    posesion=pagina[9], Estadísticas diversas=pagina[11]'''

    #si solo participa en liga  no habra ningun 'filter switcher' por lo que tamaño será None 
    if tamaño is None:
        #normalmente el tamaño contiene una lista con las competiciones en las que participo el equipo + un elemento, como el equipo solo participo en liga 
        #len(tamaño) deberia ser igual a 2 pero como es None asignaremos el valor de manera manual
        tamaño=2
    
    else:
        tamaño=len(tamaño)
    #defino que tablas voy a usar
    for i in range(1,5):
        tablas.append(pagina[(i*tamaño-(i-1))])
    for i in range(6,9):
        tablas.append(pagina[(i*tamaño-(i-1))])
    tablas.append(pagina[10*tamaño-9])

    #elimino los encabezados de las tablas y filtro a los jugadores con mas de 5 min jugados en promedio en la temporada
    for i in range(9):
        tablas[i].columns=tablas[i].columns.droplevel(level=0)
        tablas[i]=tablas[i][tablas[i]['90 s']>=5]

    
    ## BLOQUE 2
     # Lista de columnas relevantes para cada bloque
    columnas_relevantes = [
        ['Jugador', 'Posc', 'Edad', '90 s', 'PrgC', 'PrgP', 'PrgR', 'Ast', 'G-TP', 'npxG', 'xAG', 'npxG+xAG'],
        ['Jugador', 'GC90', '% Salvadas', 'PaC%'],
        ['Jugador', 'PSxG/SoT', '/90', '%deLanzamientos', 'Int.', '% de Stp', 'Núm. de OPA/90', 'DistProm.', 'Long. prom.'],
        ['Jugador', 'Dis'],
        ['Jugador', 'Int.', '% Cmp'],
        ['Jugador', 'ACT'],
        ['Jugador', 'Desp.', 'Int', 'Bloqueos', 'Tkl'],
        ['Jugador', 'Ataq. pen.', 'Toques', 'Succ'],
        ['Jugador', 'Ganados']
    ]
    dfs_arreglados=[]

    for i, df in enumerate(tablas):
        # Filtrar las columnas relevantes para este bloque
        columnas = columnas_relevantes[i]

        if i == 0:
            #elimino las columnas duplicadas
            duplicates=['G-TP','npxG','xAG','npxG+xAG']
            for duplicate in duplicates:
                df=md.drop_duplicates_columns(df,duplicate,1)

            #me quedo con las columnas que necesito
            df = df.loc[:,columnas]

            # como los apartados de Prg estan en total y no por 90min los pasare a 90 min
            Prgs=['PrgC','PrgP','PrgR']   
            for Prg in Prgs:
                df[f'{Prg}']=round(df[f'{Prg}']/df['90 s'], 2)

            

        elif i ==1:
            #me quedo con las columnas que necesito
            df = df.loc[:, columnas]
            

        elif i ==2:    
            
            df=md.drop_duplicates_columns(df,'Long. prom.',1)
            df=md.drop_duplicates_columns(df,'Int.',2)
            df=md.drop_duplicates_columns(df,'%deLanzamientos',0)
            df['Int.']=round(df['Int.']/df['90 s'],2)
            #me quedo con las columnas que necesito
            df = df.loc[:, columnas]
            
        elif i == 3:
            #cambio el apartado de Dis a Dis por 90min
            df.loc[:, 'Dis'] = round(df['Dis'] / df['90 s'], 2)
            #me quedo con las columnas que necesito
            df = df.loc[:, columnas]
            
        elif i == 4:
            #elimino las columnas duplicadas
            duplicates=['Int.','% Cmp']
            for duplicado in duplicates:
                df=md.drop_duplicates_columns(df,duplicado,0)
            df['Int.']=round(df['Int.']/df['90 s'],2)
            #me quedo con las columnas que necesito
            df = df.loc[:, columnas]
            

        elif i == 5:

            df.loc[:,'ACT']=round(df['ACT']/df['90 s'],2)
            #me quedo con las columnas que necesito
            df = df.loc[:, columnas]
            

        elif i == 6:

            df=md.drop_duplicates_columns(df,'Tkl',0)
            df['Tkl']=round(df['Tkl']/df['90 s'], 2)
            df['Bloqueos']=round(df['Bloqueos']/df['90 s'], 2)
            df['Int']=round(df['Int']/df['90 s'], 2)
            df['Desp.']=round(df['Desp.']/df['90 s'], 2)
            #me quedo con las columnas que necesito
            df = df.loc[:, columnas]
           

        elif i == 7:
            df.loc[:,'Ataq. pen.']=round(df['Ataq. pen.']/df['90 s'], 2)
            df.loc[:,'Toques']=round(df['Toques']/df['90 s'], 2)
            df.loc[:,'Succ']=round(df['Succ']/df['90 s'], 2)
            #me quedo con las columnas que necesito
            df = df.loc[:,columnas]
            

        elif i == 8:

            df.loc[:,'Ganados']=round(df['Ganados']/df['90 s'], 2)
            #me quedo con las columnas que necesito
            df = df.loc[:,columnas]

        #agrego el dataframe nuevo a la lista de dfs
        dfs_arreglados.append(df)

    ## BLOQUE 3
    resultado = reduce(lambda left, right: pd.merge(left, right, on='Jugador', how='outer'), dfs_arreglados)

    ## BLOQUE 4
    resultado=resultado.fillna(0)

    allteams=pd.read_csv(r'C:\Users\ARTUROS\Desktop\Archivos Adrian\personal\proyectos personales\lacaprichosa\dataset\All_teams.csv')


    resultado['Jugador']=int(allteams[allteams['id Web']==f'{codigoE}']['id_ml'].iloc[0])

    ## BLOQUE 5
    resultado=resultado.drop('Posc', axis=1)
    resultado=resultado.groupby('Jugador').mean().reset_index()
    resultado=resultado.rename(columns={'Jugador':'Equipo id'})

    return resultado 

In [12]:
def league_level(soup):
    #busca el nivel de la liga
    text=soup.find_all('div')[22].find('p').get_text(strip=True)
    matches = re.search(r'(\d+)° Nivel de la liga', text)
    if matches is None:
        league_level='El equipo no participa en ninguna liga profesional'
    else:
        league_level=matches.group(1)
    return league_level


In [36]:
equipito=get_players_mean(res,url)
equipito

,Equipo id,Edad,90 s,PrgC,PrgP,PrgR,Ast,G-TP,npxG,xAG,...,% Cmp,ACT,Desp.,Int,Bloqueos,Tkl,Ataq. pen.,Toques,Succ,Ganados
0,36156400,26.428571,25.452381,1.50381,3.134762,3.177143,0.096667,0.107619,0.10619,0.09619,...,79.871429,1.879524,1.615238,0.895238,0.967143,1.522857,1.725714,52.695238,0.609048,1.155714


In [37]:
equipito=get_players_mean2(res,url)
equipito

,Equipo id,Edad,90 s,PrgC,PrgP,PrgR,Ast,G-TP,npxG,xAG,...,% Cmp,ACT,Desp.,Int,Bloqueos,Tkl,Ataq. pen.,Toques,Succ,Ganados
0,36156400,26.428571,25.452381,1.50381,3.134762,3.177143,0.096667,0.107619,0.10619,0.09619,...,79.871429,1.879524,1.615238,0.895238,0.967143,1.522857,1.725714,52.695238,0.609048,1.155714


In [23]:
tamaño=2
for i in range(1,5):
    print(i*tamaño-(i-1))
for i in range(6,9):
    print(i*tamaño-(i-1))
10*tamaño-9

2
3
4
5
7
8
9


11

In [16]:
estandar[0], porteros[2], porteros_avanzada[3],tiros[4],pases[5],creacion[7],acciones_defensivas[8], posesion[9], diversas[11]

2


[   Equipo id       Edad     90 s      PrgC    PrgP    PrgR       Ast  \
 0   86022920  26.583333  21.7875  1.717083  3.0025  3.1975  0.091667   
 
        G-TP      npxG   xAG  ...  % Cmp       ACT     Desp.       Int  \
 0  0.124167  0.120417  0.09  ...   74.2  1.919167  2.308333  0.718333   
 
    Bloqueos       Tkl  Ataq. pen.     Toques      Succ  Ganados  
 0    0.8225  1.082083     1.68875  50.259583  0.689167    2.225  
 
 [1 rows x 35 columns],
 '2']

esta es la lista de columnas descrita en la documentacion

In [88]:
['Goles sin penalización',  'Goles esperados (xG) sin contar penaltis',  'Asistencias',  'xAG',  'npxG + xAG',
'Pases progresivos',  'Acarreos progresivos',  'Pases progresivos Rec',  ' Goles en contra',  'Porcentaje de salvadas',
'% Salvadas (Tiros penales)',  'Porcentaje de portería a cero',  'PSxG-GA, PSxG/SoT',  'Lanzamiento %',  'Saques de meta', 
'Prom. de longitud de los saques de meta',  'Acciones defensivas fuera del área penal',  'Distancia promedio de las acciones defensivas',
'% de cruces detenidos', 'Total de disparos', 'Pases intentados',  '% de pase completo',  'Acciones para la creación de tiros',
'Derribos',  'Intercepciones',  'Bloqueos',  'Despeje',  'Toques (Ataq. pen.)',  'Tomas exitosas',  'Toques',  'Aéreos ganados']

31

y esta es la misma lista pero con los nombres que toman en la tabla

In [ ]:
['G-TP', 'npxG',  'Ast',  'xAG',  'npxG + xAG', 'PrgP',  'PrgC',  'PrgR',  'GC90',  '% Salvadas',
'% Salvadas',  'PaC%',  '/90', 'PSxG/SoT',  'Lanzamiento %', 'Int._x', 'Long. prom.',  'Núm. de OPA/90',  'DistProm.',
'% de Stp',  'Dis', 'Int._y',  '% Cmp',  'ACT', 'Tkl',  'Int',  'Bloqueos',  'Desp.',  'Ataq. pen.',  'Succ',  'Toques',  'Ganados']

arqueros=pagina[8].head(2)
pagina[0].columns=pagina[0].columns.droplevel(level=0)
df_jugadores=pagina[0]
df_jugadores[df_jugadores['Posc']!='PO']
df_jugadores[df_jugadores['Titular']>=5]
tablaprueba=tablaprueba.drop(tablaprueba.columns[indice],axis=1)
df_jugadores

# Aca esta la logica detras de cada bloque de la funcion

## bloque 1: selector de tablas

In [ ]:
#aqui estaba calculando la sucesion para poder elegir correctamente que tablas
[0, 7, 13, 19,25,37,43,49,61]
[0, 1*tamaño-0, 2*tamaño-1,3*tamaño-2,4*tamaño-3,   6*tamaño-5,7*tamaño-6,8*tamaño-7,    10*tamaño-9 ]

elige tablas tablas[#]

In [ ]:
tamaño=len(soup.find('div', class_='filter switcher'))
tablas=[]
tablas.append(pagina[0])
for i in range(1,5):
    tablas.append(pagina[(i*tamaño-(i-1))])
for i in range(6,9):
    tablas.append(pagina[(i*tamaño-(i-1))])
tablas.append(pagina[10*tamaño-9])


for i in range(9):
    tablas[i].columns=tablas[i].columns.droplevel(level=0)
    tablas[i]=tablas[i][tablas[i]['90 s']>=5]

    

In [19]:
tablas[3]

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                 Jugador               País               Posc   
0          Ethan Horvath             us USA                 PO   
1           Amari'i Bell             jm JAM                 DF   
2            Tom Lockyer            wls WAL                 DF   
3         Elijah Adebayo            eng ENG                 DL   
4         Carlton Morris            eng ENG                 DL   
5         Allan Campbell            sct SCO                 CC   
6           Jordan Clark            eng ENG                 CC   
7             James Bree            eng ENG                 DF   
8          Alfie Doughty            eng ENG              DF,DL   
9           Gabriel Osho            eng ENG                 DF   
10  Pelly Ruddock Mpanzu             cd COD                 CC   
11             Dan Potts            eng ENG                 DF   
12           Reece Burke            eng ENG                 DF   
13     Marvelous Nakamba             zw ZIM                 CC   
14           Cody Drameh            eng ENG                 DF   
15         Sonny Bradley            eng ENG                 DF   
16        Cauley Woodrow            eng ENG              DL,CC   
17          Luke Freeman            eng ENG              CC,DL   
18         Harry Cornick            eng ENG              DL,CC   
19            Luke Berry            eng ENG                 CC   
20        Fred Onyedinma            eng ENG              DF,CC   
21          Louie Watson             ie IRL                 CC   
22        Henri Lansbury            eng ENG                 CC   
23        Cameron Jerome            eng ENG                 DL   
24        Admiral Muskwe             zw ZIM                 DL   
25         Elliot Thorpe            wls WAL              DF,CC   
26         Joseph Taylor            wls WAL                 DL   
27            James Shea            eng ENG                 PO   
28          Dion Pereira            eng ENG                 CC   
29          Harvey Isted            eng ENG                 PO   
30            Matt Macey            eng ENG                 PO   
31   Carlos Mendes Gomes             es ESP                 CC   
32        Joseph Johnson            eng ENG                 DF   
33       Joshua Williams            wls WAL                 DF   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Estándar                         ...  \
                 Edad               90 s     Gls.  Dis DaP % de TT  T/90  ...   
0                  27               48.0        0    0   0     NaN  0.00  ...   
1                  28               46.6        1   17   5    29.4  0.36  ...   
2                  27               41.3        4   30   9    30.0  0.73  ...   
3                  24               39.2        9   92  32    34.8  2.35  ...   
4                  26               38.9       20  120  42    35.0  3.08  ...   
5                  24               38.6        3   31  12    38.7  0.80  ...   
6                  28               36.8        3   60  13    21.7  1.63  ...   
7                  24               29.0        0   19   3    15.8  0.66  ...   
8                  22               26.9        2   33  11    33.3  1.23  ...   
9                  23               26.2        2   21   4    19.0  0.80  ...   
10                 29               25.1        3   23   4    17.4  0.92  ...   
11                 28               24.1        1   16   7    43.8  0.66  ...   
12                 25               17.0        2    2   2   100.0  0.12  ...   
13                 28               15.8        0    6   0     0.0  0.38  ...   
14                 20               14.4        0    6   1    16.7  0.42  ...   
15                 30               13.6        0    8   2    25.0  0.59  ...   
16                 27               12.8        3   40  15    37.5  3.13  ...   
17                 30               11.0        2   21   7    33.3  1.91  ...   
18       

## bloque 2: For principal

elimina sobrantes tablas[0]
cambia las progresiones a por 90s tablas[0]

In [ ]:
duplicates_a_eliminar=['G-TP','npxG','xAG','npxG+xAG']
columnas_eliminar0=['País','PJ','Titular','Mín','Partidos','Ass', 'Gls.', 'G+A','TP','TPint','TA','TR','xG']
tablas[0]=tablas[0].drop(columnas_eliminar0, axis=1)
for duplicate in duplicates_a_eliminar:
    tablas[0]=md.drop_duplicates_columns(tablas[0],duplicate,1)
Prgs=['PrgC','PrgP','PrgR']
for Prg in Prgs:
    tablas[0][f'{Prg}']=round(tablas[0][f'{Prg}']/tablas[0]['90 s'], 2)
tablas[0]


elimino sobrantes [1]

In [ ]:
columnas_eliminar1=['País','Posc','Edad','PJ','Titular','Mín','90 s', 'GC','DaPC','PG','PE','PP','PaC','TPint','PD','PD.1','PC','Partidos','Salvadas']
tablas[1]=tablas[1].drop(columnas_eliminar1, axis=1)

elimino sobrantes tabla porteros avanzados[2]

In [ ]:
columnas_eliminar2=['País','Posc','Edad','90 s','GC','GC.1','PD','TL','TE','GC','Partidos','PSxG','PSxG+/-','PSxG','Cmp','Int.','% Cmp','Int.',	'TI',
                    'Opp','Stp','Núm. de OPA']
tablas[2]=tablas[2].drop(columnas_eliminar2, axis=1)
tablas[2]=md.drop_duplicates_columns(tablas[2],'Long. prom.',1)


elimino sobrantes tabla tiros [3]

In [ ]:
tablas[3]['Dis']=round(tablas[3]['Dis']/tablas[3]['90 s'], 2)
tablas[3] = tablas[3].loc[:, ['Jugador','Dis']]


elimino sobrantes tabla pases [4]

In [ ]:

duplicados_a_quitar=['Int.','% Cmp']
for duplicado in duplicados_a_quitar:
    tablas[4]=md.drop_duplicates_columns(tablas[4],duplicado,0)
tablas[4]['Int.']=round(tablas[4]['Int.']/tablas[4]['90 s'],2)
tablas[4] = tablas[4].loc[:, ['Jugador','Int.','% Cmp']]

elimino sobrantes tabla [5]

elimino sobrantes tabla Creación de goles y tiros [5]

In [ ]:
tablas[5]['ACT']=round(tablas[5]['ACT']/tablas[5]['90 s'],2)
tablas[5] = tablas[5].loc[:, ['Jugador','ACT',]]
tablas[5]

tabla Acciones defensivas[6]

In [ ]:
tablas[6]=md.drop_duplicates_columns(tablas[6],'Tkl',0)
tablas[6]['Tkl']=round(tablas[6]['Tkl']/tablas[6]['90 s'], 2)
tablas[6]['Bloqueos']=round(tablas[6]['Bloqueos']/tablas[6]['90 s'], 2)
tablas[6]['Int']=round(tablas[6]['Int']/tablas[6]['90 s'], 2)
tablas[6]['Desp.']=round(tablas[6]['Desp.']/tablas[6]['90 s'], 2)
tablas[6] = tablas[6].loc[:, ['Jugador','Desp.','Int','Bloqueos','Tkl']]



In [ ]:

tablas[6]

In [ ]:
tablas[7]['Ataq. pen.']=round(tablas[7]['Ataq. pen.']/tablas[7]['90 s'], 2)
tablas[7]['Toques']=round(tablas[7]['Toques']/tablas[7]['90 s'], 2)
tablas[7]['Succ']=round(tablas[7]['Succ']/tablas[7]['90 s'], 2)
tablas[7] = tablas[7].loc[:, ['Jugador','Ataq. pen.','Toques','Succ']]
tablas[7]




tabla Estadísticas diversas [8]

In [ ]:
tablas[8]

In [ ]:
tablas[8]['Ganados']=round(tablas[8]['Ganados']/tablas[8]['90 s'], 2)
tablas[8] = tablas[8].loc[:, ['Jugador','Ganados']]
tablas[8]

## bloque 3: unificador de tablas

In [ ]:
temp=tablas[0]
i=0
while i < 8:
    resultado = pd.merge(temp,tablas[i+1], on='Jugador', how='outer')
    temp=resultado
    i+=1

resultado

In [ ]:
resultado.columns

In [16]:
codigo='b8fd03ef'

## bloque 4: asignador de codigo

In [22]:
texto=soup.find_all('div')[22].find('p').get_text(strip=True)
matches = re.search(r'(\d+)° Nivel de la liga', texto)
nivelliga=matches.group(1)

if nivelliga!='1':
    allteams=pd.read_csv(r'C:\Users\ARTUROS\Desktop\Archivos Adrian\personal\proyectos personales\lacaprichosa\dataset\All_teamsB.csv')
else:
    allteams=pd.read_csv(r'C:\Users\ARTUROS\Desktop\Archivos Adrian\personal\proyectos personales\lacaprichosa\dataset\All_teamsA.csv')

C:\Users\ARTUROS\AppData\Local\Temp\ipykernel_9272\592938234.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  equipitoBrandy['Jugador']=int(allteams[allteams['id Web']==f'{codigo}']['id Local'])


,Jugador,Posc,Edad,90 s,PrgC,PrgP,PrgR,Ast,G+A-TP,xG+xAG,...,% Cmp,ACT,Desp.,Int,Bloqueos,Tkl,Ataq. pen.,Toques,Succ,Ganados
0,12,CC,26,49.6,1.98,7.00,0.87,0.14,0.22,0.23,...,91.3,2.24,1.05,1.05,1.03,1.83,1.53,88.00,0.69,1.75
1,12,PO,28,47.5,0.00,0.00,0.00,0.02,0.02,0.01,...,85.5,0.04,0.32,0.02,0.00,0.06,0.00,35.28,0.02,0.17
2,12,DL,22,45.7,0.96,1.53,3.98,0.20,1.16,1.13,...,72.4,2.01,0.37,0.11,0.37,0.09,6.04,21.23,0.50,1.47
3,12,CC,31,42.7,1.80,5.76,3.91,0.14,0.40,0.38,...,87.7,2.93,0.44,0.77,0.61,0.98,2.79,62.20,0.63,0.66
4,12,DF,27,42.4,0.80,3.77,0.59,0.02,0.05,0.08,...,92.7,1.01,1.25,0.64,0.97,1.49,1.11,74.20,0.17,0.78
5,12,CC,31,40.7,3.19,7.25,7.44,0.69,0.93,0.69,...,72.2,5.36,0.37,0.37,0.76,0.91,2.83,57.96,1.03,0.34
6,12,"DL,CC",27,40.1,3.32,4.64,6.98,0.15,0.32,0.23,...,86.2,2.82,0.27,0.75,0.85,1.57,3.39,62.37,1.35,0.17
7,12,DL,26,38.6,5.39,3.65,11.01,0.28,0.41,0.47,...,84.4,4.12,0.39,0.36,0.70,0.85,5.08,49.92,1.84,0.47
8,12,DF,25,38.5,0.60,3.58,0.13,0.00,0.03,0.11,...,93.3,0.68,2.10,0.75,0.91,1.22,1.01,85.53,0.29,1.71
9,12,"DL,CC",31,34.0,3.03,2.68,8.97,0.38,0.71,0.66,...,83.0,4.38,0.18,0.38,0.82,1.15,4.03,52.21,1.09,0.29


## bloque 5: Agrupacion de resultados

In [18]:
resultado=resultado.drop('Posc', axis=1)
resultado=resultado.groupby('Jugador').mean().reset_index()
resultado=resultado.rename(columns={'Jugador':'Equipo id'})

12    12
Name: id Local, dtype: int64